## Offline Data Processing

In [1]:
# including the project directory to the notebook level
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
# import modules
from alumni_scripts import offline_data_process as odp
import json

In [2]:
with open('../auths.json', 'r') as fp:
    api_args = json.load(fp)

time_args = {
    'start_year': 2018,
    'start_month': 8,
    'start_day': 1,
    'start_hour': 0,
    'start_minute': 0,
    'start_second': 0,
    'end_year': 2018,
    'end_month': 8,
    'end_day': 1,
    'end_hour': 0,
    'end_minute': 30,
    'end_second': 0,
    'trend_id': '2681',
    'save_path':'../data/raw_data/alumni_data.csv'
}
api_args.update(time_args)

odp.pull_data(**api_args)